# ABBA cell count analysis

This notebook is the last step in the ABBA whole-brain cell counting analysis.  
It assumes you have done the following steps:
- Alignment of brain slices in ABBA, exported to a QuPath project.
- Detected cells of interest in QuPath. The detections should be exported to ```.csv``` files (one per slice) in a folder called ```results```. 
- If there are regions to exclude, you should have drawn them and exported to ```.txt``` files (one per slice) in a folder called ```regions_to_exclude```.

Run this notebook to load the cell counts and do analysis on them. 

## Before we start ...
The majority of the functions and classes we need written in 3 files: ```brain_hierarchy.py```, ```readCSV_helpers.py``` and ```pls_helpers.py```. We will now import the necessary functions and classes from these python files to this notebook, so that we can use them later:

In [1]:
from brain_hierarchy import AllenBrainHierarchy
from readCSV_helpers import collect_and_analyze_cell_counts,save_results
from pls_helpers import PLS

And we'll need other python functions to easily read and manipulate data and make nice plots:

In [2]:
import pandas as pd
# import copy
# import json
import numpy as np
import os

# import matplotlib.pyplot as plt
import plotly.express as px
# import seaborn as sns
# import pickle
import plotly.graph_objects as go

## The Allen Brain Atlas

We start by importing the mouse Allen Brain Atlas, in which we find information about all brain regions (their parent region and children regions in the brain hierarchy, for example).

In [3]:
path_to_allen_json = "./data/AllenMouseBrainOntology.json"
AllenBrain = AllenBrainHierarchy(path_to_allen_json) 

edges = AllenBrain.edges_dict
tree = AllenBrain.tree_dict
brain_region_dict = AllenBrain.brain_region_dict
regions = list(brain_region_dict.keys())

We now have access to useful information about all brain regions. Below, show the first three of them:

In [4]:
AllenBrain.df.head(3)

,atlas_id,ontology_id,acronym,region_name,color_hex_triplet,graph_order,st_level,hemisphere_id,parent_structure_id,children,id,distance_from_root
id,,,,,,,,,,,,
997,-1.0,1,root,root,FFFFFF,0,0,3,None,"[{'id': 8, 'atlas_id': 0, 'ontology_id': 1, 'a...",997,0
8,0.0,1,grey,Basic cell groups and regions,BFDAE3,1,1,3,997,"[{'id': 567, 'atlas_id': 70, 'ontology_id': 1,...",8,1
1009,691.0,1,fiber tracts,fiber tracts,CCCCCC,1101,1,3,997,"[{'id': 967, 'atlas_id': 686, 'ontology_id': 1...",1009,1


We can also visualize the hierarchy of brain regions as a network (a tree). **Note that running the above cell may take a few minutes**.

In [5]:
# Plot brain region hierarchy
# If you want to plot it, install PyDot (pydot)
# fig = AllenBrain.plot_plotly_graph()
# fig.show()

Based on the graph above, you might want to specify the regions on which you want to do further analysis:  
*Note: to see more information about the regions, hover over them with your mouse.*

- Specify a level. Analysis can only be done one one level (slice) in the brain region.

- To exclude brain regions that belong to a certain branch, add the *abbreviated* nodes at the beginning of the branches to the list above.  
Example:  
```branches_to_exclude = ['retina', 'VS']```  
means that **all the subregions that belong to the retina and the ventricular systems** are excluded from the analysis.

In [6]:
level = 6
branches_to_exclude = ['retina','VS','grv','fiber tracts']

Now, get the selected regions as a variable:

In [7]:
selected_regions = AllenBrain.list_regions_to_analyze(level, branches_to_exclude)
print(f'You selected %d regions to analyze.'%len(selected_regions))

You selected 288 regions to analyze.


## Load data

Now, we're ready to read the ```.csv``` files with the cell counts, and also the exclusion files (if there were regions to exclude).  
Below, you have to specify:
- ```root```: Absolute path to the folder that contains the animal folders.
- ```animal_list_bla```: A list of names of the folders corresponding to animals in the BLA group. Indeed, it is necessary to store the results in individual folders for each animal.
- ```animal_list_il```: A list of names of the folders corresponding to animals in the IL group.
- ```sum_dict```: A [dictionary](https://www.dataquest.io/blog/python-dictionaries/) which specifies how to sum columns in the original dataframe.  

Example: ```sum_dict = {'RAB': 'Num Rabies', 'Num CTB: Rabies', 'Num Rabies: TVA', 'Num CTB: Rabies: TVA'}``` 
means that the total number of Rabies+ neurons is the sum of single positives (Rabies), double positves (Rabies+TVA and Rabies+CTB) and triple positives (Rabies+TVA+CTB).


 Due gruppi: Stress + contro
Programma fa tutto su RAB -> Solo Num AF647.
Provare a modificar per ottenere densita in mm^2 (da micron)

In [8]:
# ####################################### SET PARAMETERS ####################################



root = './data/results/'

'''
Originally we run the code in three cases: RvsC, RvsS, SvsC
As they were just two groups in each case, I've left the original names animal_list_bla and animal_list_it.
I did so not to modify too much the code, as these variable were referenced several times and I didn't want to break the code
'''
# current:
animal_list_bla = ['Stress_5S', 'Stress_8S', 'Stress_10S', 'Stress_13S', 'Resilient_1R', 'Resilient_2R', 'Resilient_3R', 'Resilient_4R', 'Resilient_11R']
animal_list_il = ['Control_17C', 'Control_18C', 'Control_19C']

# old
# animal_list_bla = ['Stress_5S', 'Stress_8S','Stress_13S', 'Stress_14S', 'Stress_10S', 'Resilient_1R', 'Resilient_2R', 'Resilient_3R', 'Resilient_11R']
# animal_list_il = ['Control_19C', 'Control_17C', 'Control_16C', 'Control_18C']

sum_dict = {'area': 'Area um^2',
            'CTB': ['Num Detections'],     #I've set all the columns to Num Detections, to not leave them empty
            'RAB': ['Num AF647'],          #The calculations in the prrogram are made on RAB, so I set it to our variable of interest Num AF467
            'TVA': ['Num Detections'],
            'CTB_RAB': ['Num Detections'],
            'CTB_TVA': ['Num Detections'],
            'RAB_TVA': ['Num Detections'],
            'CTB_RAB_TVA': 'Num Detections'
            }
# ###########################################################################################

Now, we load the BLA and IL results seperately in two pandas dataframes, and save the results. Note that regions to exclude are automatically excluded.

In [9]:
print(sum_dict)

{'area': 'Area um^2', 'CTB': ['Num Detections'], 'RAB': ['Num AF647'], 'TVA': ['Num Detections'], 'CTB_RAB': ['Num Detections'], 'CTB_TVA': ['Num Detections'], 'RAB_TVA': ['Num Detections'], 'CTB_RAB_TVA': 'Num Detections'}


In [10]:
# Load cell counts
results_bla = collect_and_analyze_cell_counts(root, animal_list_bla, AllenBrain, sum_dict=sum_dict)
results_il = collect_and_analyze_cell_counts(root, animal_list_il, AllenBrain, sum_dict=sum_dict)

# Save results
output_path = os.path.join(root, 'results')
save_results(results_il, output_path, 'results_cell_counts_C.txt')
save_results(results_bla, output_path, 'results_cell_counts_S.txt')

Importing slices in Stress_5S...
Imported 32 slices.

Raw cell counts are saved to ./data/results/Stress_5S/results_python
Importing slices in Stress_8S...
Imported 48 slices.

Raw cell counts are saved to ./data/results/Stress_8S/results_python
Importing slices in Stress_10S...
Imported 39 slices.

Raw cell counts are saved to ./data/results/Stress_10S/results_python
Importing slices in Stress_13S...
Imported 31 slices.

Raw cell counts are saved to ./data/results/Stress_13S/results_python
Importing slices in Resilient_1R...
Imported 41 slices.

Raw cell counts are saved to ./data/results/Resilient_1R/results_python
Importing slices in Resilient_2R...
Imported 45 slices.

Raw cell counts are saved to ./data/results/Resilient_2R/results_python
Importing slices in Resilient_3R...
Imported 44 slices.

Raw cell counts are saved to ./data/results/Resilient_3R/results_python
Importing slices in Resilient_4R...
Imported 43 slices.

Raw cell counts are saved to ./data/results/Resilient_4R/res

True

The data are stored in ```results_il``` and ```results_bla```:

# Partial Least Squares  

The analysis done below is taken from the tutorial written by [Krishnan et al.](https://www.sciencedirect.com/science/article/pii/S1053811910010074).  
Run the 2 cells below to get started.

In [11]:
# PLS
animal_list = animal_list_bla + animal_list_il
normalization = 'Density'   #Normalize on Density rather then Percentage
rank = 1

# Create a PLS object
rabiesPLS = PLS(results_il, results_bla, animal_list_il, animal_list_bla, selected_regions, 'RAB', normalization)

# Show the matrix X
rabiesPLS.X

/home/carlo/Documents/Neuroinformatics/fear-extinction/CFos_analysis/pls_helpers.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  self.X = data.loc[regions].T.dropna('columns')
/home/carlo/Documents/Neuroinformatics/fear-extinction/CFos_analysis/pls_helpers.py:50: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  Y = pd.get_dummies(y)
/home/carlo/Documents/Neuroinformatics/fear-extinction/CFos_analysis/pls_helpers.py:46: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  self.Ly = pd.get_dummies(self.y) @ self.u


,CLA,LA,PA,MEV,SCO,CUN,VTN,PPN,MO,SS,...,CS,LDT,PRNr,RPO,CN,NTB,RM,CENT3,"CUL4, 5",ANcr1
Control_17C,0.000395,0.000294,0.000352,0.000255,0.000133,0.000172,0.000061,0.000081,0.000372,0.000745,...,0.00014,0.000293,0.000099,0.000246,0.00025,0.000641,0.000445,0.000367,0.00026,0.000568
Control_18C,0.000031,0.000045,0.000028,0.000064,0.00002,0.000029,0.000011,0.000026,0.000027,0.000059,...,0.000031,0.000054,0.000037,0.000009,0.000009,0.000017,0.00001,0.000022,0.000014,0.0
Control_19C,0.00003,0.000069,0.000024,0.000118,0.000147,0.000144,0.0,0.000185,0.00017,0.000216,...,0.000327,0.000296,0.000184,0.00042,0.000209,0.000214,0.000258,0.000315,0.000285,0.000128
Stress_5S,0.000243,0.000131,0.000006,0.000129,0.0,0.000059,0.000036,0.000065,0.000101,0.000133,...,0.000066,0.000134,0.000072,0.000063,0.000063,0.000378,0.00017,0.000035,0.000046,0.000031
Stress_8S,0.000528,0.000259,0.000315,0.000337,0.00052,0.000242,0.000478,0.000326,0.000622,0.000846,...,0.000292,0.000347,0.00029,0.000262,0.000308,0.000411,0.000356,0.000299,0.000251,0.000346
Stress_10S,0.000274,0.000227,0.000207,0.00058,0.000035,0.000295,0.000091,0.000282,0.000469,0.0007,...,0.000204,0.000393,0.000099,0.000283,0.000221,0.0,0.000145,0.000498,0.000359,0.000234
Stress_13S,0.000279,0.000052,0.000001,0.000221,0.000944,0.000075,0.000193,0.000098,0.000083,0.000194,...,0.000099,0.000335,0.00011,0.000113,0.000169,0.000273,0.000117,0.000497,0.000272,0.000157
Resilient_1R,0.000186,0.000117,0.000101,0.000339,0.00047,0.000117,0.00011,0.000089,0.00018,0.000334,...,0.000084,0.000166,0.000064,0.000103,0.000175,0.00007,0.000077,0.000151,0.000166,0.000229
Resilient_2R,0.000119,0.000168,0.000187,0.00033,0.000224,0.000153,0.000034,0.000124,0.000048,0.000111,...,0.000146,0.000133,0.000089,0.000072,0.000046,0.000021,0.000059,0.000045,0.000068,0.000105
Resilient_3R,0.000048,0.000068,0.000051,0.0,0.000162,0.00006,0.000031,0.000111,0.000029,0.000079,...,0.000089,0.000183,0.000094,0.000406,0.00001,0.000042,0.000116,0.0,0.000026,0.000024


In [12]:
# Show the matrix Y
pd.get_dummies(rabiesPLS.y).rename(columns={0:'IL',1:'BLA'})

/tmp/ipykernel_907/927763439.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pd.get_dummies(rabiesPLS.y).rename(columns={0:'IL',1:'BLA'})


,IL,BLA
Control_17C,0,1
Control_18C,0,1
Control_19C,0,1
Stress_5S,1,0
Stress_8S,1,0
Stress_10S,1,0
Stress_13S,1,0
Resilient_1R,1,0
Resilient_2R,1,0
Resilient_3R,1,0


The two matrices printed above (X and Y) illustrate the data on which the PLS is done.  
- ```X:``` The rows in this matrix are the mice. The columns in the matrix are the regions selected for analysis. The values in the matrix are the **percentage of Rabies+ cells in that region relative to the whole brain.**
- ```Y:``` The rows in this matrix are the mice. The columns in the matrix are the 2 groups (IL or BLA). **A value in this matrix is 1 if the mice belongs to the specified group**.

In brief, PLS analyzes the relationship (correlation) between the columns of ```X``` and ```Y```. In our specific case, there will be 2 important outputs:
- **Salience scores**: Each brain region has a salience score. A high salience scores means that the brain region explains much of the correlation between ```X``` and ```Y```.  
- **Singular values**: These are the eigenvalues of the correlation matrix $R = Y^TX$.

## Random permutations to see whether we can differentiate signal from noise. 
Here, we randomly shuffle the group to which a mouse belongs, and calculate the singular values of the permuted dataset.  
From [Krishnan et al.](https://www.sciencedirect.com/science/article/pii/S1053811910010074):  
> The set of all the (permuted) singular values provides a sampling distribution of the singular values under the null hypothesis and, therefore can be used as a null hypothesis test.

*Note: running the cell below will take a few minutes.*

In [13]:
num_permutations = 5000
print(f'Randomly permuting singular values %d times ...'%num_permutations)
s,singular_values = rabiesPLS.randomly_permute_singular_values(num_permutations)
print('Done!\n')

Randomly permuting singular values 5000 times ...
Done!



In [14]:
# TODO: move to Plotly

# Plot distribution of singular values
plt.figure(figsize=(10,4))
plt.hist(singular_values[:,0],bins=10)
plt.axvline(rabiesPLS.s[0], color='r')
plt.xlabel('First singular value')
plt.ylabel('Frequency')
plt.legend([f'Experiment','Sampling distribution\nunder H0 (%d permutations)'%num_permutations])
plt.show()

NameError: name 'plt' is not defined

In [14]:
# Calculate p-value = Probability(experiment | H0)
p = (singular_values[:,0] > s[0]).sum() / num_permutations
print('p-value = '+str(p))

p-value = 0.5204


## Bootstrap to identify stable salience scores

Here, we use [bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) (= sampling of the mice in the dataset, with replacement) to get an estimate of which salience scores are stable.

From [Krishnan et al.](https://www.sciencedirect.com/science/article/pii/S1053811910010074):  
> When a vector of saliences is considered generalizable and is kept for further analysis, we need to identify its elements that are stable through resampling. In practice, the stability of an element is evaluated by dividing it by its standard error. [...] To estimate the standard errors, we create bootstrap samples which are obtained by sampling with replacement the observations in and (Efron and Tibshirani, 1986). A salience standard error is then estimated as the standard error of the saliences from a large number of these bootstrap samples (say 1000 or 10000). **The ratios are akin to a Z-score, therefore when they are larger than 2 the corresponding saliences are considered significantly stable.**

*Note: Running the cell below will take a few minutes.*

In [15]:
num_bootstrap = 5000
print(f'Bootstrapping salience scores %d times ...'%num_bootstrap)
u_salience_scores,v_salience_scores = rabiesPLS.bootstrap_salience_scores(rank,num_bootstrap)
print('Done!')

Bootstrapping salience scores 5000 times ...


/home/carlo/Documents/Neuroinformatics/fear-extinction/CFos_analysis/pls_helpers.py:50: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  Y = pd.get_dummies(y)
/home/carlo/Documents/Neuroinformatics/fear-extinction/CFos_analysis/pls_helpers.py:50: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  Y = pd.get_dummies(y)
/home/carlo/Documents/Neuroinformatics/fear-extinction/CFos_analysis/pls_helpers.py:50: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  Y = pd.get_dummies(y)
/home/carlo/Documents/Neuroinformatics/fear-extinction/CFos_analysis/pls_helpers.py:50: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (ma

Done!


/home/carlo/Documents/Neuroinformatics/fear-extinction/CFos_analysis/pls_helpers.py:50: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  Y = pd.get_dummies(y)
/home/carlo/Documents/Neuroinformatics/fear-extinction/CFos_analysis/pls_helpers.py:50: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  Y = pd.get_dummies(y)
/home/carlo/Documents/Neuroinformatics/fear-extinction/CFos_analysis/pls_helpers.py:50: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  Y = pd.get_dummies(y)
/home/carlo/Documents/Neuroinformatics/fear-extinction/CFos_analysis/pls_helpers.py:50: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (ma

In [17]:
output_path

'./data/results/results'

In [16]:
# Plot PLS salience scores
plot_threshold = 1.2 # Only brain regions with a salience higher than plot_threshold are shown. 2 is the significance threshold.

file_title = 'PLS_RAB' + '_' + normalization + '.png'
output_path = os.path.join(root, 'results_python')

tp, salient_regions = rabiesPLS.plot_salience_scores(plot_threshold, output_path, file_title, brain_region_dict,
                              fig_width=1000, fig_height=2000)

##### salient_regions.reset_index()['index']

In [18]:
df = salient_regions.reset_index()
df.columns = ['region', 'salience']
df['salience'] = df['salience'].abs()
df = df.sort_values(by='salience')
df.to_csv('./data/R_results/salient_regions.csv', sep=';', index=False)
df

,region,salience
7,HIP,1.207125
38,SCsg,1.236325
48,TRN,1.244756
5,RSP,1.298118
9,BMAa,1.311599
13,AAA,1.320639
6,PTLp,1.358118
8,BLAa,1.359128
16,GPi,1.404495
14,IA,1.408889


In [19]:
pls_filename = 'PLS_RAB_' + normalization + '_salience_scores.txt'
save_results(v_salience_scores.rename(columns={0:'salience score'}), output_path, pls_filename)


! A results_python folder already existed in root. I am overwriting previous results!

Results are saved in ./data/results/results_python

Done!


True

# Plot percentages

In [20]:
results_il[('RAB','Percentage')].to_csv('plot.csv', index=True)

In [21]:
# In this case we wanted to normalize it based on the density, rather then the Percentage 
# I didn't modify the various labels in the plot as I was just focused on adapting the code to our dataset, rather then polishing it

tracer_to_plot = 'RAB'
normalization = 'Density' # 'Density','Percentage','RelativeDensity'
threshold = 1e-2 # Only plot bars with value larger than threshold (1e-6, 1e-2, 3)
y_axis_label = 'region_names' # change this to 'acronym' to have acronyms on the y-axis

# Calculate mean values
data_to_plot_il = pd.DataFrame(results_il[(tracer_to_plot,normalization)].rename('cell counts'))
mean_data_il = data_to_plot_il.reset_index().groupby('level_0').mean()
sem_data_il = data_to_plot_il.reset_index().groupby('level_0').sem()

data_to_plot_bla = pd.DataFrame(results_bla[(tracer_to_plot,normalization)].rename('cell counts'))
mean_data_bla = data_to_plot_bla.reset_index().groupby('level_0').mean()
sem_data_bla = data_to_plot_bla.reset_index().groupby('level_0').sem()

# Determine which regions to plot  
mean_sum = mean_data_il + mean_data_bla
#regs_to_plot = mean_sum[(mean_sum['cell counts']>threshold) & (mean_sum['cell counts'].notnull())].sort_values(by='cell counts').index.to_list()
regs_to_plot = rabiesPLS.X.columns.to_list()

# y-axis, with seperate values for each region
y_axis_il, ticklabels = pd.factorize(data_to_plot_il.loc[regs_to_plot].reset_index()['level_0'])
y_axis_bla, ticklabels = pd.factorize(data_to_plot_bla.loc[regs_to_plot].reset_index()['level_0'])
if(y_axis_label=='region_names'):
    ticklabels = [AllenBrain.brain_region_dict[reg] for reg in ticklabels]
     
fig = go.Figure()

# Barplot
fig.add_trace(go.Bar(
                     x = mean_data_il.loc[regs_to_plot]['cell counts'],
                     name = 'C mean',
                     error_x = dict(
                         type='data',
                         array=sem_data_il.loc[regs_to_plot]['cell counts']
                     )
              )
)

fig.add_trace(go.Bar(
                     x = mean_data_bla.loc[regs_to_plot]['cell counts'],
                     name = 'S mean',
                     error_x = dict(
                         type='data',
                         array=sem_data_bla.loc[regs_to_plot]['cell counts']
                     )
              )
)

fig.update_layout(barmode='group', colorway=['rgb(0,255,0)', 'rgb(255,0,0)'])

# Scatterplot (animals)
fig.add_trace(go.Scatter(
                    mode = 'markers',
                    y = y_axis_il - 0.2,
                    x = data_to_plot_il.loc[regs_to_plot]['cell counts'],
                    name = 'C animals',
                    opacity=0.5,
                    marker=dict(
                        color='rgb(0,255,0)',
                        size=5,
                        line=dict(
                            color='rgb(0,0,0)',
                            width=1
                        )
                    )
              )
)

fig.add_trace(go.Scatter(
                    mode = 'markers',
                    y = y_axis_bla + 0.2,
                    x = data_to_plot_bla.loc[regs_to_plot]['cell counts'],
                    name = 'S animals',
                    opacity=0.5,
                    marker=dict(
                        color='rgb(255,0,0)',
                        size=5,
                        line=dict(
                            color='rgb(0,0,0)',
                            width=1
                        )
                    )
              )
)

# Figure title
title = ''
if normalization=='RelativeDensity':
    title = '['+tracer_to_plot+ '(r) / area(r)] / ['+tracer_to_plot+'(brain) / area(brain)].'
elif normalization=='Density':
    title = '['+tracer_to_plot+ '(r) / area(r)]'
elif normalization=='Percentage':
    title = '['+tracer_to_plot+ '(r) / brain(r)]'

# Update layout
fig.update_layout(
    title = title,
    yaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(0,len(regs_to_plot)),
        ticktext = ticklabels
    ),
    xaxis=dict(
        title = 'Cfos density (relative to brain)'
    ),
    width=900, height=5000,
    hovermode="x unified",
    yaxis_range = [-1,len(regs_to_plot)+1]
)

fig.show()

# Save figure as PNG
output_path = os.path.join(root, 'results_python')
if not(os.path.exists(output_path)):
    os.mkdir(output_path)
file_title = 'barplot_' + tracer_to_plot + '_' + normalization + 'CvS.png'
output_file = os.path.join(output_path, file_title)
fig.write_image(output_file)

/tmp/ipykernel_907/987503279.py:11: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/tmp/ipykernel_907/987503279.py:12: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sem is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/tmp/ipykernel_907/987503279.py:15: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/tmp/ipykernel_907/987503279.py:16: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sem is deprecated. In a future version, numeric_only will default to F